# Example of Use

These commands were used to generate the output data in [LAMMPS Simulation Data of Alchemical Processes](https://data.nist.gov/od/id/mds2-3637).

## Import Packages

In [1]:

import numpy as np
import generate_alchemical_lammps_inputs as gali


## Lennard-Jones Dimer

A dimer made of two Lennard-Jones beads is solvated at T*=0.75. Here we generate a section of the LAMMPS input file to compute the solvation free energy of the dimer using MBAR. Note that this output will also contain the information to calculate the solvation free energy with thermodynamic integration also.

In [5]:
# Simple input where lambda pair changes
output = gali.generate_mbar_input(
    [1.0, 0.0], # The parameter will change from 1 to 0
    -0.2, # The parameter will be reduced by 0.2 between each step, so there will be 5 windows
    [
        [ # This list represents an adaptation, more than one may be listed (e.g., change charge and nonbonded at the same time although that isn't recommended)
            "pair", # May be "pair" or "atom", where charges can only be changed as per atom properties
            (
                "lj/cut/soft", # Pair style as defined in lammps
                "lambda", # Define the changing parameter to be "lambda", an option for this pair style in the LAMMPS documentation (https://docs.lammps.org/fix_adapt_fep.html)
                1, # Solvent bead type (or range)
                2  # Sulute bead type (or range)
            )
        ]
    ], 
    0.75, # Temperature
#    output_file="your/output/here", # Optional file to write to
    n_run_equil_steps=1000000, # Duration of equilibration run for each step (i.e., window)
    n_run_prod_steps=5000000,  # Duration of production run for each step (i.e., window)
)
for line in output:
    print(line[:-2])


# Variables and System Condition
variable TK equal 0.7
variable freq equal 100
variable runtime_equil equal 100000
variable runtime_prod equal 500000
variable pinst equal pres
variable tinst equal tem
variable vinst equal vo
variable pe equal p
variable deltacdm equal 0.01 # delta used in central different method for derivative in T
variable deltacdm2 equal -0.01 # delta used in central different method for derivative in T
variable nblocks equal 
variable lambdas vector [1.0,0.8,0.6,0.4,0.2,0.0

thermo ${freq

# Set-up Loo
variable runid loop 1 ${nblocks} pa
    label runloop

    # Adjust param for the box and equilibrat
    variable param equal v_lambdas[v_runid
    if "${runid} == 1" then 
        "jump SELF skipequil
    variable ind equal v_runid-
    variable param0 equal v_lambdas[v_ind
    variable paramramp equal ramp(v_param0,v_param
    fix ADAPT all adapt/fep 1 
        pair lj/cut/soft lambda 1 2  v_paramramp
    thermo_style custom step v_paramramp  temp press pe evdwl e

## Solvated Benzene

### Step 1: Scale Charges of Benzene to Zero

The package LAMMPS cannot remove the Coulombic interactions between some atoms (e.g., benzene and water) while allowing other to remain (e.g., H in benzene with C in benzene). We must then scale the charges of the entire molecule to zero.

In [6]:
# Input where charges change
output = gali.generate_mbar_input(
    [1.0, 0.0], # Scale charges from full strength to none
    -0.2, # Decrease in steps of 0.2 (5 windows)
    [
        ["atom", ("charge", 3, -0.115000)], # Change the atom property, charge, of atom type 3, with a full strength value of -0.115000
        ["atom", ("charge", 4, 0.115000)], # Change another atom property...
    ],
    300, # Temperature in K
#    output_file="your/output/here", # Optional file to write to
    n_run_equil_steps=1000000, # Duration of equilibration run for each step (i.e., window)
    n_run_prod_steps=5000000,  # Duration of production run for each step (i.e., window)
)
for line in output:
    print(line[:-2])


# Variables and System Condition
variable TK equal 30
variable freq equal 100
variable runtime_equil equal 100000
variable runtime_prod equal 500000
variable pinst equal pres
variable tinst equal tem
variable vinst equal vo
variable pe equal p
variable deltacdm equal 0.01 # delta used in central different method for derivative in T
variable deltacdm2 equal -0.01 # delta used in central different method for derivative in T
variable nblocks equal 
variable lambdas vector [1.0,0.8,0.6,0.4,0.2,0.0
variable deltacdmA0 equal -0.115*v_deltacd
variable deltacdm2A0 equal 0.115*v_deltacd
variable deltacdmA1 equal 0.115*v_deltacd
variable deltacdm2A1 equal -0.115*v_deltacd

thermo ${freq

# Set-up Loo
variable runid loop 1 ${nblocks} pa
    label runloop

    # Adjust param for the box and equilibrat
    variable param equal v_lambdas[v_runid
    if "${runid} == 1" then 
        "jump SELF skipequil
    variable ind equal v_runid-
    variable param0 equal v_lambdas[v_ind
    variable paramramp 

### Step 2a: Remove Non-bonded Interaction between Benzene and Water without Trajectory

Because saving the trajectories for a free energy calculation can be quite large, this is the preferred method. However, if later you determine that you need an additional value of lambda, you'll need to rerun the entire calculation.

In [7]:
# Input where pair changes after charges change
output = gali.generate_mbar_input(
    [1.0, 0.0], 
    -0.0625, # Remove within 16 steps
    [
        [
            "pair", # change a pair style (in the previous step we changed an atom property)
            (
                "lj/cut/soft",
                "lambda", 
                "1*2", # Range of atom types for water (in format required by LAMMPS)
                "3*4" # Range of atom types for the solute, benzene (in format required by LAMMPS)
            )
        ]
    ],
    300,
    parameter_array=np.array([1,0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6,0.5,0.4,0.3,0.2,0.1,0.05,0.0]), # Overwrite parameter_change with custom lambda spacing
    fix_adapt_changes2 = [ # Keep the change in charges done in the previous step, so defined again here
        ["atom", ("charge", 3, -0.115000)],
        ["atom", ("charge", 4, 0.115000)],   
    ],
    parameter2_value=0, # Define the final scaling factor for the parameter change in step 1
#    output_file="your/output/here", # Optional file to write to
    n_run_equil_steps=1000000, # Duration of equilibration run for each step (i.e., window)
    n_run_prod_steps=5000000,  # Duration of production run for each step (i.e., window)
)
for line in output:
    print(line[:-2])


# Variables and System Condition
variable TK equal 30
variable freq equal 100
variable runtime_equil equal 100000
variable runtime_prod equal 500000
variable pinst equal pres
variable tinst equal tem
variable vinst equal vo
variable pe equal p
variable deltacdm equal 0.01 # delta used in central different method for derivative in T
variable deltacdm2 equal -0.01 # delta used in central different method for derivative in T
variable nblocks equal 1
variable lambdas vector [1.0,0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6,0.5,0.4,0.3,0.2,0.1,0.05,0.0

# Set Previous Chang
variable param2 equal 
variable param2A0 equal -0.115*v_param
variable param2A1 equal 0.115*v_param
fix ADAPT2 all adapt/fep 1 
    atom charge 3  v_param2A0 
    atom charge 4  v_param2A1

thermo ${freq

# Set-up Loo
variable runid loop 1 ${nblocks} pa
    label runloop

    # Adjust param for the box and equilibrat
    variable param equal v_lambdas[v_runid
    if "${runid} == 1" then 
        "jump SELF skipequil
    variable

### Step 2b: Remove Non-bonded Interaction between Benzene and Water with Trajectory

Generate the input file to write the trajectories needed to calculate the free energy of solvation. In the next step, we generate the input to calculate the values needed for MBAR FOR ONE WINDOW using rerun and the trajectories. This might be desirable if you don't know the number of lambda windows or spacing that your system requires.

In [8]:
# Input where pair changes after charges change
ouptut = gali.generate_traj_input(
    [1.0, 0.0], 
    -0.0625, 
    [
        ["pair", ("lj/cut/soft", "lambda", "1*2", "3*4")]
    ],
    300,
    parameter_array=np.array([1,0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6,0.5,0.4,0.3,0.2,0.1,0.05,0.0]), # Overwrite parameter_change with custom lambda spacing
    fix_adapt_changes2 = [
        ["atom", ("charge", 3, -0.115000)],
        ["atom", ("charge", 4, 0.115000)],   
    ],
    parameter2_value=0,
#    output_file="your/output/here", # Optional file to write to
    n_run_equil_steps=1000000, # Duration of equilibration run for each step (i.e., window)
    n_run_prod_steps=5000000,  # Duration of production run for each step (i.e., window)
)
for line in output:
    print(line[:-2])


# Variables and System Condition
variable TK equal 30
variable freq equal 100
variable runtime_equil equal 100000
variable runtime_prod equal 500000
variable pinst equal pres
variable tinst equal tem
variable vinst equal vo
variable pe equal p
variable deltacdm equal 0.01 # delta used in central different method for derivative in T
variable deltacdm2 equal -0.01 # delta used in central different method for derivative in T
variable nblocks equal 1
variable lambdas vector [1.0,0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6,0.5,0.4,0.3,0.2,0.1,0.05,0.0

# Set Previous Chang
variable param2 equal 
variable param2A0 equal -0.115*v_param
variable param2A1 equal 0.115*v_param
fix ADAPT2 all adapt/fep 1 
    atom charge 3  v_param2A0 
    atom charge 4  v_param2A1

thermo ${freq

# Set-up Loo
variable runid loop 1 ${nblocks} pa
    label runloop

    # Adjust param for the box and equilibrat
    variable param equal v_lambdas[v_runid
    if "${runid} == 1" then 
        "jump SELF skipequil
    variable

In [9]:
output = gali.generate_rerun_mbar(
    0.5, # Value of lambda to make a rerun script for
    [1.0, 0.0], 
    -0.0625, 
    [
        ["pair", ("lj/cut/soft", "lambda", "1*2", "3*4")]
    ],
    300,
    parameter_array=np.array([1,0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6,0.5,0.4,0.3,0.2,0.1,0.05,0.0]),
    fix_adapt_changes2 = [
        ["atom", ("charge", 3, -0.115000)],
        ["atom", ("charge", 4, 0.115000)],   
    ],
    parameter2_value=0,
#    output_file="your/output/here", # Optional file to write to
)
for line in output:
    print(line[:-2])


# Variables and System Condition
variable TK equal 30
variable freq equal 100
variable pinst equal pres
variable tinst equal tem
variable vinst equal vo
variable pe equal p
variable param equal 0.
variable lambdas vector [1.0,0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6,0.5,0.4,0.3,0.2,0.1,0.05,0.0

thermo ${freq

# Set Previous Chang
variable param2 equal 
variable param2A0 equal -0.115*v_param
variable param2A1 equal 0.115*v_param
fix ADAPT2 all adapt/fep 1 
    atom charge 3  v_param2A0 
    atom charge 4  v_param2A1

# Initialize compute
fix ADAPT all adapt/fep ${freq} 
    pair lj/cut/soft lambda 1*2 3*4  v_param 
variable delta0 equal 0.
compute FEP000 all fep ${TK} 
    pair lj/cut/soft lambda 1*2 3*4  v_delta0 
    volume ye
variable param000 equal v_param+v_delta
fix FEPout000 all ave/time ${freq} 1 ${freq} v_param v_param000 v_param2 v_pe 
    c_FEP000[1] c_FEP000[2] c_FEP000[3] file files/mbar_lj-cut-soft-lambda_${param}_${param000}_charge--0.0.txt
variable delta1 equal 0.4
compute 

### Step 3: Restore the Charges of Benzene Atoms in Vacuum

Because the previous two steps were missing the intramolecular charge interactions of benzene, and the end of the second system resulted in benzene being completely decoupled from water, we must now determine the free energy of returning the charges to complete the cycle.

In [10]:
# Input where charges change
output = gali.generate_mbar_input(
    [0.0, 1.0], # Scale the charges from zero to in full force
    0.2, 
    [
        ["atom", ("charge", 1, -0.115000)],
        ["atom", ("charge", 2, 0.115000)],
    ],
    300, 
#    output_file="your/output/here", # Optional file to write to
    n_run_equil_steps=1000000, # Duration of equilibration run for each step (i.e., window)
    n_run_prod_steps=5000000,  # Duration of production run for each step (i.e., window)
)
for line in output:
    print(line[:-2])


# Variables and System Condition
variable TK equal 30
variable freq equal 100
variable runtime_equil equal 100000
variable runtime_prod equal 500000
variable pinst equal pres
variable tinst equal tem
variable vinst equal vo
variable pe equal p
variable deltacdm equal 0.01 # delta used in central different method for derivative in T
variable deltacdm2 equal -0.01 # delta used in central different method for derivative in T
variable nblocks equal 
variable lambdas vector [0.0,0.2,0.4,0.6,0.8,1.0
variable deltacdmA0 equal -0.115*v_deltacd
variable deltacdm2A0 equal 0.115*v_deltacd
variable deltacdmA1 equal 0.115*v_deltacd
variable deltacdm2A1 equal -0.115*v_deltacd

thermo ${freq

# Set-up Loo
variable runid loop 1 ${nblocks} pa
    label runloop

    # Adjust param for the box and equilibrat
    variable param equal v_lambdas[v_runid
    if "${runid} == 1" then 
        "jump SELF skipequil
    variable ind equal v_runid-
    variable param0 equal v_lambdas[v_ind
    variable paramramp 